# 🏥 Cohort Fairness Analysis Framework

This notebook demonstrates how to analyze cohort fairness in your distillation experiments.

## Overview
- Analyze performance differences across patient cohorts
- Evaluate if distillation affects cohort fairness  
- Generate comprehensive visualizations and reports
- Provide clinical insights and recommendations

## Cohorts in OhioT1DM Dataset
- Different patient study groups or time periods
- May represent enrollment waves or study phases
- Could affect prediction performance due to temporal factors

## Setup and Imports

In [ ]:
# Import required libraries
import sys
sys.path.append('../')  # Add parent directory to path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import json

# Import our cohort fairness analyzer
from cohort_fairness_analyzer import CohortFairnessAnalyzer

# Set up visualization style
plt.style.use('default')
sns.set_palette("husl")

print("🏥 Cohort Fairness Analysis System Loaded!")
print("🎯 Ready to analyze cohort fairness in your diabetes prediction models")

# Initialize analyzer
analyzer = CohortFairnessAnalyzer()
print(f"📊 Patient data loaded: {len(analyzer.patient_data)} patients")

# Quick cohort overview
cohort_counts = {}
for patient_id, cohort in analyzer.patient_data.items():
    cohort_counts[cohort] = cohort_counts.get(cohort, 0) + 1

print(f"📈 Cohort Distribution: {cohort_counts}")

## 1. Cohort Demographics Analysis

In [ ]:
# Load patient data from CSV to get full demographics
data_df = pd.read_csv("/workspace/LLM-TIME/data/ohiot1dm/data.csv")

print("📊 COHORT DEMOGRAPHICS")
print("=" * 50)

# Cohort distribution
cohort_distribution = data_df['Cohort'].value_counts()
total_patients = len(data_df)

print(f"Total patients: {total_patients}")
print("Cohort Distribution:")
for cohort, count in cohort_distribution.items():
    percentage = (count / total_patients) * 100
    print(f"  {cohort}: {count} patients ({percentage:.1f}%)")

# Visualize cohort distribution
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Cohort pie chart
colors = plt.cm.Set3(np.linspace(0, 1, len(cohort_distribution)))
cohort_distribution.plot(kind='pie', ax=ax1, autopct='%1.1f%%', colors=colors)
ax1.set_title('Cohort Distribution')
ax1.set_ylabel('')

# Cohort by gender if available
if 'Gender' in data_df.columns:
    cohort_gender = pd.crosstab(data_df['Cohort'], data_df['Gender'])
    cohort_gender.plot(kind='bar', ax=ax2, color=['lightblue', 'lightpink'])
    ax2.set_title('Cohort Distribution by Gender')
    ax2.set_xlabel('Cohort')
    ax2.set_ylabel('Count')
    ax2.legend(title='Gender')
    ax2.tick_params(axis='x', rotation=45)

# Cohort by age group if available
if 'Age Group' in data_df.columns:
    cohort_age = pd.crosstab(data_df['Cohort'], data_df['Age Group'])
    cohort_age.plot(kind='bar', ax=ax3, colormap='viridis')
    ax3.set_title('Cohort Distribution by Age Group')
    ax3.set_xlabel('Cohort')
    ax3.set_ylabel('Count')
    ax3.legend(title='Age Group')
    ax3.tick_params(axis='x', rotation=45)

# Cohort by pump model if available
if 'Pump Model' in data_df.columns:
    cohort_pump = pd.crosstab(data_df['Cohort'], data_df['Pump Model'])
    cohort_pump.plot(kind='bar', ax=ax4, color=['lightgreen', 'lightcoral'])
    ax4.set_title('Cohort Distribution by Pump Model')
    ax4.set_xlabel('Cohort')
    ax4.set_ylabel('Count')
    ax4.legend(title='Pump Model')
    ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\n📋 Patient Data Preview:")
display(data_df.head())

# Show cohort characteristics
print(f"\n🔍 COHORT CHARACTERISTICS:")
cohort_stats = data_df.groupby('Cohort').agg({
    'Patient ID': 'count',
    'Age Group': lambda x: x.mode().iloc[0] if not x.empty else 'Unknown',
    'Gender': lambda x: f"{(x=='Female').sum()}F/{(x=='Male').sum()}M"
}).round(2)
cohort_stats.columns = ['Patient Count', 'Most Common Age Group', 'Gender Split']
display(cohort_stats)

## 2. Load Distillation Experiment Results

In [ ]:
# Run cohort fairness analysis
print("🔍 RUNNING COHORT FAIRNESS ANALYSIS")
print("=" * 50)

try:
    # Analyze latest experiment
    cohort_groups = analyzer.analyze_latest()
    
    if cohort_groups:
        print("✅ Cohort analysis complete!")
        
        # Show results summary
        print(f"\n📊 Results Summary:")
        for cohort, patients in cohort_groups.items():
            if patients:
                avg_mse = np.mean([p['mse'] for p in patients])
                avg_mae = np.mean([p['mae'] for p in patients])
                avg_rmse = np.mean([p['rmse'] for p in patients])
                
                print(f"\n🏥 {cohort}:")
                print(f"   Patient count: {len(patients)}")
                print(f"   Average MSE: {avg_mse:.6f}")
                print(f"   Average MAE: {avg_mae:.6f}")
                print(f"   Average RMSE: {avg_rmse:.6f}")
    else:
        print("❌ No cohort results found!")
        
except Exception as e:
    print(f"❌ Error during analysis: {e}")
    print("Make sure you have run some distillation experiments first!")

## 3. Cohort Fairness Metrics

In [ ]:
if 'cohort_groups' in locals() and cohort_groups:
    # Calculate comprehensive fairness metrics
    cohort_performance = {}
    
    print("⚖️ COHORT FAIRNESS ANALYSIS")
    print("=" * 50)
    
    # Calculate performance for each cohort
    for cohort, patients in cohort_groups.items():
        if patients:
            mse_values = [p['mse'] for p in patients]
            mae_values = [p['mae'] for p in patients]
            rmse_values = [p['rmse'] for p in patients]
            
            cohort_performance[cohort] = {
                'count': len(patients),
                'avg_mse': np.mean(mse_values),
                'avg_mae': np.mean(mae_values),
                'avg_rmse': np.mean(rmse_values),
                'std_mse': np.std(mse_values),
                'std_mae': np.std(mae_values),
                'std_rmse': np.std(rmse_values),
                'patients': patients
            }
    
    # Calculate fairness ratios if we have multiple cohorts
    if len(cohort_performance) >= 2:
        # Find best and worst performing cohorts
        cohorts = list(cohort_performance.keys())
        cohort_rmse = [(cohort, perf['avg_rmse']) for cohort, perf in cohort_performance.items()]
        cohort_rmse.sort(key=lambda x: x[1])
        
        best_cohort = cohort_rmse[0]
        worst_cohort = cohort_rmse[-1]
        
        # Calculate fairness metrics
        rmse_ratio = worst_cohort[1] / best_cohort[1]
        rmse_diff = worst_cohort[1] - best_cohort[1]
        
        print(f"🎯 Fairness Analysis Across {len(cohorts)} Cohorts:")
        print(f"   Best performing: {best_cohort[0]} (RMSE: {best_cohort[1]:.6f})")
        print(f"   Worst performing: {worst_cohort[0]} (RMSE: {worst_cohort[1]:.6f})")
        print(f"   Performance Difference: {rmse_diff:.6f}")
        print(f"   Performance Ratio: {rmse_ratio:.2f}x")
        
        # Fairness assessment
        if rmse_ratio <= 1.1:
            fairness_level = "EXCELLENT"
            fairness_color = "🟢"
        elif rmse_ratio <= 1.2:
            fairness_level = "GOOD" 
            fairness_color = "🟡"
        elif rmse_ratio <= 1.5:
            fairness_level = "MODERATE"
            fairness_color = "🟠"
        else:
            fairness_level = "POOR"
            fairness_color = "🔴"
            
        print(f"\n{fairness_color} Fairness Assessment: {fairness_level}")
        
        # Temporal analysis if cohorts represent time periods
        print(f"\n📅 TEMPORAL CONSIDERATIONS:")
        cohorts_sorted = sorted(cohorts)
        if len(cohorts_sorted) >= 3:
            print(f"   Cohort progression: {' → '.join(cohorts_sorted)}")
            # Check if performance improves or degrades over time
            temporal_rmse = [(cohort, cohort_performance[cohort]['avg_rmse']) for cohort in cohorts_sorted]
            
            trend_direction = "improving" if temporal_rmse[-1][1] < temporal_rmse[0][1] else "degrading"
            print(f"   Performance trend: {trend_direction} over cohorts")
        
        # Store results for later use
        fairness_results = {
            'best_cohort': best_cohort[0],
            'worst_cohort': worst_cohort[0],
            'rmse_ratio': rmse_ratio,
            'rmse_diff': rmse_diff,
            'fairness_level': fairness_level,
            'cohorts': cohorts_sorted
        }
    
    else:
        print("⚠️ Need at least 2 cohorts for fairness comparison")
        fairness_results = None

## 4. Performance Visualization Dashboard

In [ ]:
if 'cohort_performance' in locals() and cohort_performance:
    # Create comprehensive visualization dashboard
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Average Performance Comparison
    cohorts = list(cohort_performance.keys())
    rmse_values = [cohort_performance[cohort]['avg_rmse'] for cohort in cohorts]
    colors = plt.cm.Set3(np.linspace(0, 1, len(cohorts)))
    
    bars1 = ax1.bar(cohorts, rmse_values, color=colors, alpha=0.7)
    ax1.set_xlabel('Cohort')
    ax1.set_ylabel('Average RMSE')
    ax1.set_title('Average RMSE by Cohort')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars1, rmse_values):
        ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + bar.get_height()*0.01,
                f'{value:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # 2. Patient Count Distribution
    patient_counts = [cohort_performance[cohort]['count'] for cohort in cohorts]
    ax2.pie(patient_counts, labels=cohorts, autopct='%1.0f patients', 
            colors=colors, startangle=90)
    ax2.set_title('Patient Distribution by Cohort')
    
    # 3. Performance Distribution (Box Plot)
    rmse_data = []
    cohort_labels = []
    
    for cohort in cohorts:
        patients = cohort_performance[cohort]['patients']
        rmse_values = [p['rmse'] for p in patients]
        rmse_data.extend(rmse_values)
        cohort_labels.extend([cohort] * len(rmse_values))
    
    # Create DataFrame for seaborn
    plot_df = pd.DataFrame({'Cohort': cohort_labels, 'RMSE': rmse_data})
    sns.boxplot(data=plot_df, x='Cohort', y='RMSE', ax=ax3)
    ax3.set_title('RMSE Distribution by Cohort')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, alpha=0.3)
    
    # 4. Fairness Summary
    ax4.text(0.1, 0.9, 'COHORT FAIRNESS SUMMARY', fontsize=14, fontweight='bold')
    
    if 'fairness_results' in locals() and fairness_results:
        summary_text = [
            f"Cohorts Analyzed: {len(cohorts)}",
            f"",
            f"Performance Ranking:",
            f"  Best: {fairness_results['best_cohort']}",
            f"  Worst: {fairness_results['worst_cohort']}",
            f"",
            f"Fairness Metrics:",
            f"  Performance Ratio: {fairness_results['rmse_ratio']:.2f}x",
            f"  Assessment: {fairness_results['fairness_level']}",
            f"",
            f"Total Patients: {sum(patient_counts)}"
        ]
    else:
        summary_text = [
            f"Cohorts Found: {len(cohorts)}",
            f"Total Patients: {sum(patient_counts)}",
            "",
            "Need multiple cohorts",
            "for fairness comparison"
        ]
    
    for i, text in enumerate(summary_text):
        ax4.text(0.1, 0.8 - i*0.08, text, fontsize=11, transform=ax4.transAxes)
    
    ax4.set_xlim(0, 1)
    ax4.set_ylim(0, 1)
    ax4.axis('off')
    
    plt.tight_layout()
    plt.show()

## 5. Temporal Analysis and Trends

In [ ]:
if 'cohort_performance' in locals() and len(cohort_performance) >= 2:
    # Analyze temporal patterns in cohort performance
    print("📅 TEMPORAL COHORT ANALYSIS")
    print("=" * 50)
    
    # Sort cohorts assuming they have temporal order
    cohorts_sorted = sorted(cohort_performance.keys())
    
    # Create temporal performance plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. Performance trend over cohorts
    rmse_trend = [cohort_performance[cohort]['avg_rmse'] for cohort in cohorts_sorted]
    patient_trend = [cohort_performance[cohort]['count'] for cohort in cohorts_sorted]
    
    ax1.plot(range(len(cohorts_sorted)), rmse_trend, 'bo-', linewidth=2, markersize=8)
    ax1.set_xlabel('Cohort Sequence')
    ax1.set_ylabel('Average RMSE')
    ax1.set_title('Performance Trend Across Cohorts')
    ax1.set_xticks(range(len(cohorts_sorted)))
    ax1.set_xticklabels(cohorts_sorted, rotation=45)
    ax1.grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(range(len(rmse_trend)), rmse_trend, 1)
    p = np.poly1d(z)
    ax1.plot(range(len(rmse_trend)), p(range(len(rmse_trend))), "r--", alpha=0.8)
    
    # 2. Sample size trend
    ax2.bar(range(len(cohorts_sorted)), patient_trend, alpha=0.7, color='skyblue')
    ax2.set_xlabel('Cohort Sequence')
    ax2.set_ylabel('Patient Count')
    ax2.set_title('Sample Size Across Cohorts')
    ax2.set_xticks(range(len(cohorts_sorted)))
    ax2.set_xticklabels(cohorts_sorted, rotation=45)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Analyze trends
    print(f"\n📈 TREND ANALYSIS:")
    
    # Performance trend analysis
    if len(rmse_trend) >= 2:
        trend_slope = z[0]  # From linear fit above
        if abs(trend_slope) < 0.001:
            trend_desc = "STABLE - consistent performance across cohorts"
        elif trend_slope < 0:
            trend_desc = "IMPROVING - performance gets better in later cohorts"
        else:
            trend_desc = "DECLINING - performance gets worse in later cohorts"
        
        print(f"   Performance Trend: {trend_desc}")
        print(f"   Trend Slope: {trend_slope:.6f} RMSE per cohort")
    
    # Sample size analysis
    avg_patients = np.mean(patient_trend)
    std_patients = np.std(patient_trend)
    cv_patients = std_patients / avg_patients if avg_patients > 0 else 0
    
    print(f"   Sample Size Consistency: {cv_patients:.2f} coefficient of variation")
    if cv_patients < 0.2:
        print("   → Consistent sample sizes across cohorts")
    else:
        print("   → Variable sample sizes - may affect fairness assessment")
    
    # Store temporal results
    temporal_results = {
        'trend_slope': float(trend_slope) if len(rmse_trend) >= 2 else 0,
        'trend_description': trend_desc if len(rmse_trend) >= 2 else "Unknown",
        'sample_size_cv': float(cv_patients),
        'cohorts_sequence': cohorts_sorted
    }
else:
    print("⚠️ Need at least 2 cohorts for temporal analysis")
    temporal_results = None

## 6. Individual Patient Performance Analysis

In [ ]:
if 'cohort_groups' in locals() and cohort_groups:
    # Analyze individual patient performance
    all_patients = []
    
    for cohort, patients in cohort_groups.items():
        for patient in patients:
            all_patients.append({
                'Patient ID': patient['patient_id'],
                'Cohort': cohort,
                'MSE': patient['mse'],
                'MAE': patient['mae'],
                'RMSE': patient['rmse']
            })
    
    if all_patients:
        patient_df = pd.DataFrame(all_patients)
        
        print("👤 INDIVIDUAL PATIENT PERFORMANCE")
        print("=" * 50)
        display(patient_df)
        
        # Create individual performance visualizations
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # 1. Scatter plot of individual performance by cohort
        cohort_colors = plt.cm.Set3(np.linspace(0, 1, len(cohort_groups)))
        
        for i, cohort in enumerate(cohort_groups.keys()):
            cohort_data = patient_df[patient_df['Cohort'] == cohort]
            ax1.scatter(cohort_data.index, cohort_data['RMSE'], 
                       label=cohort, alpha=0.7, s=100, color=cohort_colors[i])
        
        ax1.set_xlabel('Patient Index')
        ax1.set_ylabel('RMSE')
        ax1.set_title('Individual Patient RMSE by Cohort')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. Performance comparison violin plot
        sns.violinplot(data=patient_df, x='Cohort', y='RMSE', ax=ax2)
        ax2.set_title('RMSE Distribution by Cohort')
        ax2.tick_params(axis='x', rotation=45)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Performance statistics
        print("\n📊 PERFORMANCE STATISTICS BY COHORT:")
        stats = patient_df.groupby('Cohort')['RMSE'].agg(['mean', 'std', 'min', 'max', 'count'])
        display(stats)
        
        # Outlier analysis
        print("\n🔍 OUTLIER ANALYSIS:")
        overall_mean = patient_df['RMSE'].mean()
        overall_std = patient_df['RMSE'].std()
        threshold = overall_mean + 2 * overall_std
        
        outliers = patient_df[patient_df['RMSE'] > threshold]
        if not outliers.empty:
            print(f"   Patients with RMSE > {threshold:.4f} (mean + 2*std):")
            for _, patient in outliers.iterrows():
                print(f"   • {patient['Patient ID']} ({patient['Cohort']}): {patient['RMSE']:.4f}")
        else:
            print("   No significant outliers detected (RMSE > mean + 2*std)")

## 7. Clinical and Research Insights

In [ ]:
if 'cohort_performance' in locals() and len(cohort_performance) >= 2:
    print("🔬 COHORT CLINICAL AND RESEARCH INSIGHTS")
    print("=" * 60)
    
    # Cohort interpretation insights
    print("🏥 COHORT ANALYSIS:")
    print("   • Cohorts may represent different study phases or enrollment periods")
    print("   • Different time periods may have varying data quality or protocols")
    print("   • Temporal effects on glucose monitoring technology evolution")
    print("   • Patient populations may differ between enrollment waves")
    
    if 'fairness_results' in locals() and fairness_results:
        print(f"\n⚖️ FAIRNESS ASSESSMENT:")
        if fairness_results['rmse_ratio'] <= 1.1:
            print("✅ EXCELLENT cohort fairness!")
            print("   • Models perform equally well across all study cohorts")
            print("   • No cohort-specific bias detected")
            print("   • Consistent model performance over time/groups")
        elif fairness_results['rmse_ratio'] <= 1.2:
            print("👍 GOOD cohort fairness with minor differences")
            print("   • Acceptable performance variation between cohorts")
            print("   • Minor temporal or group effects present")
        else:
            print("⚠️ SIGNIFICANT cohort fairness issues detected!")
            print("   • Performance varies notably between cohorts")
            print("   • May indicate temporal drift or population differences")
        
        print(f"\n🎯 RECOMMENDATIONS:")
        
        if fairness_results['rmse_ratio'] <= 1.2:
            print("✅ Current approach is working well:")
            print("   • Continue unified model training across cohorts")
            print("   • Monitor fairness in future longitudinal studies")
            print("   • Document temporal robustness as model strength")
        else:
            print("🛠️ IMPROVEMENT ACTIONS NEEDED:")
            print("   • Investigate cohort differences in detail")
            print(f"   • Collect more training data for {fairness_results['worst_cohort']}")
            print("   • Consider temporal domain adaptation techniques")
            print("   • Implement cohort-aware model architecture")
        
        # Temporal-specific insights
        if 'temporal_results' in locals() and temporal_results:
            print(f"\n📅 TEMPORAL INSIGHTS:")
            print(f"   Performance trend: {temporal_results['trend_description']}")
            
            if temporal_results['trend_slope'] < -0.001:
                print("   🎯 POSITIVE: Model performance improves over time")
                print("   • Later cohorts benefit from refined protocols")
                print("   • Consider using temporal features in modeling")
            elif temporal_results['trend_slope'] > 0.001:
                print("   ⚠️ CONCERN: Model performance degrades over time")
                print("   • May indicate dataset shift or protocol changes")
                print("   • Consider temporal adaptation strategies")
            else:
                print("   ✅ STABLE: Consistent performance across time")
                print("   • Good temporal generalizability")
        
        print(f"\n💡 CLINICAL CONSIDERATIONS:")
        print("   • Cohorts may reflect:")
        print("     - Different enrollment periods")
        print("     - Technology upgrades between study phases")
        print("     - Seasonal or environmental factors")
        print("     - Patient population changes")
        print("   • Consider cohort as a feature for model training")
        print("   • Monitor for temporal distribution shift")
        print("   • Validate model performance prospectively")

elif len(cohort_performance) == 1:
    print("📊 SINGLE COHORT ANALYSIS:")
    cohort_name = list(cohort_performance.keys())[0]
    print(f"   Only {cohort_name} patients in dataset")
    print("   Cannot assess cohort fairness")
    print("   Consider investigating cohort definitions")
else:
    print("⚠️ No cohort performance data available")

## 8. Save Results and Generate Report

In [ ]:
# Save comprehensive results
if 'cohort_performance' in locals() and cohort_performance:
    print("💾 SAVING COHORT FAIRNESS RESULTS")
    print("=" * 50)
    
    # Prepare results for saving
    results_summary = {
        'analysis_type': 'cohort_fairness',
        'timestamp': pd.Timestamp.now().isoformat(),
        'cohorts_analyzed': list(cohort_performance.keys()),
        'total_patients': sum([perf['count'] for perf in cohort_performance.values()]),
        'cohort_performance': cohort_performance
    }
    
    # Add fairness metrics if available
    if 'fairness_results' in locals() and fairness_results:
        results_summary['fairness_metrics'] = fairness_results
    
    # Add temporal analysis if available
    if 'temporal_results' in locals() and temporal_results:
        results_summary['temporal_analysis'] = temporal_results
    
    # Convert numpy types for JSON serialization
    def convert_numpy(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, np.number):
            return float(obj)
        elif isinstance(obj, dict):
            return {key: convert_numpy(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy(item) for item in obj]
        return obj
    
    results_summary = convert_numpy(results_summary)
    
    # Save to file
    output_file = "cohort_fairness_results.json"
    with open(output_file, 'w') as f:
        json.dump(results_summary, f, indent=2)
    
    print(f"✅ Results saved to: {output_file}")
    
    # Display final summary
    print(f"\n📋 FINAL SUMMARY:")
    print(f"   Cohorts analyzed: {len(cohort_performance)}")
    print(f"   Total patients: {results_summary['total_patients']}")
    if 'fairness_metrics' in results_summary:
        print(f"   Fairness ratio: {results_summary['fairness_metrics']['rmse_ratio']:.2f}x")
        print(f"   Fairness level: {results_summary['fairness_metrics']['fairness_level']}")
    if 'temporal_analysis' in results_summary:
        print(f"   Temporal trend: {results_summary['temporal_analysis']['trend_description']}")
    
else:
    print("⚠️ No results to save - run the analysis first!")

## 9. Next Steps and Integration

### If Cohort Fairness is Excellent (≤1.2x ratio):
- ✅ **Great job!** Your model is cohort-fair
- 📊 Continue monitoring fairness in longitudinal studies  
- 🔄 Use this as evidence of temporal robustness
- 📝 Document this fairness strength in your research

### If Cohort Fairness Needs Improvement (>1.2x ratio):
- 🔍 **Investigation**: Understand what cohorts represent temporally
- 📚 **Data Collection**: Gather more data for underperforming cohorts
- 🛠️ **Domain Adaptation**: Implement temporal domain adaptation
- ⚖️ **Fairness Constraints**: Use temporal fairness training (see `Fairness_Integration_Guide.ipynb`)

### Additional Analyses:
- 👫 **Gender Fairness**: Run `Gender_Fairness_Analysis.ipynb`
- 🎂 **Age Fairness**: Check `Age_Group_Fairness_Analysis.ipynb`
- 📱 **Pump Model Fairness**: Analyze `Pump_Model_Fairness_Analysis.ipynb`
- 📡 **Sensor Fairness**: Review `Sensor_Band_Fairness_Analysis.ipynb`
- 🏆 **Comprehensive**: Use `Legendary_Fairness_Analysis.ipynb` for all features

### Temporal Considerations:
- 📅 **Temporal Features**: Consider adding time-based features
- 🔄 **Continuous Learning**: Implement online learning for new cohorts
- 🧪 **Prospective Validation**: Test model on future patient cohorts
- 📈 **Trend Monitoring**: Set up automated fairness monitoring

### Technical Integration:
- 🔧 See `fairness/integration_guide.py` for implementation
- 📖 Review temporal domain adaptation techniques
- 🧪 Experiment with cohort-stratified training approaches
- 🔬 Investigate cohort definitions and temporal factors